## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=notebooks/quickstarts%2Fno2-map-plot.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Fetch STAC item for a particular date and collection - NO2 
   2. Pass STAC item in to the raster API `/stac/tilejson.json` endpoint
   3. Visualize tiles using `folium`


In [1]:
import requests
import folium

## Declare your collection of interest

You can discover available collections the following ways:

* Programmatically: see example in the `list-collections.ipynb` notebook
* JSON API: https://staging-stac.delta-backend.com/collections
* STAC Browser: http://veda-staging-stac-browser.s3-website-us-west-2.amazonaws.com

In [2]:
STAC_API_URL = "https://staging-stac.delta-backend.com"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

collection_name = "no2-monthly"

## Fetch STAC collection

We will use `requests` to fetch all the metadata about the collection of interest from STAC.

In [3]:
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'}],
 'title': 'NO₂',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01 00:00:00+00',
     '2023-09-30 00:00:00+00']]}},
 'license': 'MIT',
 'keywords': None,
 'providers': None,
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2023-09-30T00:00:00Z']},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of NO₂ and less acti

## Fetch STAC item for a particular time

We can use the search API to find the item that matches exactly our time of interest.

In [6]:
response = requests.post(
    f"{STAC_API_URL}/search",
    json={
        "collections": [collection_name],
        "query": {"datetime": {"eq": "2021-01-01T00:00:00"}},
        "limit": 100,
    },
).json()
items = response["features"]
len(items)

1

Let's take a look at that one item.

In [7]:
item = items[0]
item

{'id': 'OMI_trno2_0.10x0.10_202101_Col3_V4.nc',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items/OMI_trno2_0.10x0.10_202101_Col3_V4.nc'}],
 'assets': {'cog_default': {'href': 's3://veda-data-store-staging/no2-monthly/OMI_trno2_0.10x0.10_202101_Col3_V4.nc.tif',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'roles': ['data', 'layer'],
   'title': 'Default COG Layer',
   'description': 'Cloud optimized default layer to display on map',
   'raster:bands

In [8]:
item_stats = item['assets']['cog_default']['raster:bands'][0]['statistics']
rescale_values = item_stats['minimum'], item_stats['maximum']

## Use `/stac/tilejson.json` to get tiles

We pass the, item id, collection name, and the `rescale_values` in to the RASTER API endpoint and get back a tile.

In [10]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={item['collection']}&item={item['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=rdbu_r"
    f"&rescale={rescale_values[0]},{rescale_values[1]}",
).json()
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_0.10x0.10_202101_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=rdbu_r&rescale=-4107596126486528.0%2C35781585143857150'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

With that tile url in hand we can create a simple visualization using `folium`.

In [11]:
folium.Map(
    tiles=tiles["tiles"][0],
    attr="VEDA",
)